<a href="https://colab.research.google.com/github/bilalProgTech/btech-nmims/blob/master/NN-DL/Lab-Work/20230929-Lab-8-2-BTech-NNDL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
os.environ['KAGGLE_CONFIG_DIR'] = '/content'
!kaggle datasets download -d fanconic/skin-cancer-malignant-vs-benign

 94% 305M/325M [00:01<00:00, 185MB/s]
100% 325M/325M [00:01<00:00, 214MB/s]


In [ ]:
!unzip skin-cancer-malignant-vs-benign.zip

In [ ]:
!pip install fastai==1.0.61

In [4]:
import pandas as pd
import numpy as np
import tensorflow as tf

from IPython.display import Image, display

import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings('ignore')

In [5]:
from fastai.imports import *
from fastai import *
from fastai.vision import *

In [6]:
tfms = get_transforms(do_flip = True,
                      max_rotate = 40)

data = ImageDataBunch.from_folder('/content/train',
                                  valid_pct = 0.2,
                                  bs = 32,
                                  size = 256).normalize(imagenet_stats)

In [7]:
learn_resnet101 = cnn_learner(data, models.resnet101, metrics=accuracy)
learn_resnet101.fit(epochs = 10)

Downloading: "https://download.pytorch.org/models/resnet101-63fe2227.pth" to /root/.cache/torch/hub/checkpoints/resnet101-63fe2227.pth
100%|██████████| 171M/171M [00:01<00:00, 143MB/s]


epoch,train_loss,valid_loss,accuracy,time
0,0.597524,0.331022,0.848197,00:43
1,0.375145,0.349595,0.838710,00:30
2,0.280483,0.322396,0.876660,00:32
3,0.209920,0.315870,0.878558,00:32
4,0.190954,0.381270,0.842505,00:32
5,0.134502,0.471976,0.867173,00:32
6,0.126451,0.362243,0.882353,00:31
7,0.101694,0.416432,0.867173,00:32
8,0.083272,0.468589,0.870968,00:31
9,0.061500,0.514596,0.869070,00:32


In [8]:
learn_resnet50 = cnn_learner(data, models.resnet50, metrics=accuracy)
learn_resnet50.fit(epochs = 10)

Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 268MB/s]


epoch,train_loss,valid_loss,accuracy,time
0,0.593774,0.355944,0.853890,00:21
1,0.361171,0.291646,0.869070,00:21
2,0.292544,0.373965,0.870968,00:21
3,0.220478,0.394836,0.889943,00:21
4,0.180334,0.339916,0.876660,00:22
5,0.153156,0.351477,0.876660,00:22
6,0.141032,0.359208,0.884250,00:22
7,0.122799,0.397529,0.889943,00:22
8,0.103723,0.388167,0.886148,00:21
9,0.082683,0.408540,0.891841,00:21


In [ ]:
learn_resnet152 = cnn_learner(data, models.resnet152, metrics=accuracy)
learn_resnet152.fit(epochs = 10)

In [9]:
learn_densenet161 = cnn_learner(data, models.densenet161, metrics=accuracy)
learn_densenet161.fit(epochs = 10)

Downloading: "https://download.pytorch.org/models/densenet161-8d451a50.pth" to /root/.cache/torch/hub/checkpoints/densenet161-8d451a50.pth
100%|██████████| 110M/110M [00:00<00:00, 188MB/s] 


epoch,train_loss,valid_loss,accuracy,time
0,0.582514,0.328235,0.836812,00:56
1,0.316336,0.314404,0.870968,00:45
2,0.228265,0.334742,0.878558,00:46
3,0.157279,0.453607,0.880455,00:45
4,0.125324,0.366438,0.874763,00:45
5,0.100184,0.450006,0.874763,00:45
6,0.079664,0.457446,0.878558,00:45
7,0.064070,0.500962,0.882353,00:46
8,0.038483,0.499193,0.891841,00:46
9,0.057641,0.560564,0.888046,00:46


In [10]:
img = open_image('/content/train/benign/100.jpg')
learn_densenet161.predict(img)[0]

Category tensor(0)

In [11]:
img = open_image('/content/train/malignant/10.jpg')
learn_densenet161.predict(img)[0]

Category tensor(1)

In [12]:
files = []
target = []
for dirname, _, filenames in os.walk('/content/test'):
    for filename in filenames:
        filepath = os.path.join(dirname, filename)
        files.append(filepath)
        target.append(filepath.split('/')[-2])
test = pd.DataFrame({'files': files, 'target': target})
test = test.sample(frac=1)
test = test.reset_index(drop=True)
test.head()

,files,target
0,/content/test/malignant/1297.jpg,malignant
1,/content/test/malignant/139.jpg,malignant
2,/content/test/benign/1797.jpg,benign
3,/content/test/malignant/1027.jpg,malignant
4,/content/test/malignant/575.jpg,malignant


In [15]:
def predict_benign_or_malignant(model_learn, filepath):
    img = open_image(filepath)
    return model_learn.predict(img)[0]

In [16]:
test['resnet50'] = test['files'].apply(lambda x: predict_benign_or_malignant(learn_resnet50, x))
test['resnet50'] = test['resnet50'].astype('str')

In [17]:
test.head()

,files,target,resnet50
0,/content/test/malignant/1297.jpg,malignant,malignant
1,/content/test/malignant/139.jpg,malignant,malignant
2,/content/test/benign/1797.jpg,benign,benign
3,/content/test/malignant/1027.jpg,malignant,malignant
4,/content/test/malignant/575.jpg,malignant,malignant


In [18]:
test['densenet161'] = test['files'].apply(lambda x: predict_benign_or_malignant(learn_densenet161, x))
test['densenet161'] = test['densenet161'].astype('str')
test.head()

,files,target,resnet50,densenet161
0,/content/test/malignant/1297.jpg,malignant,malignant,malignant
1,/content/test/malignant/139.jpg,malignant,malignant,malignant
2,/content/test/benign/1797.jpg,benign,benign,benign
3,/content/test/malignant/1027.jpg,malignant,malignant,benign
4,/content/test/malignant/575.jpg,malignant,malignant,malignant


In [19]:
from sklearn.metrics import accuracy_score, confusion_matrix

In [20]:
accuracy_score(test['target'], test['resnet50'])

0.8984848484848484

In [21]:
confusion_matrix(test['target'], test['resnet50'])

array([[326,  34],
       [ 33, 267]])

In [22]:
accuracy_score(test['target'], test['densenet161'])

0.8818181818181818

In [23]:
confusion_matrix(test['target'], test['densenet161'])

array([[314,  46],
       [ 32, 268]])

In [ ]:
learn_vgg16 = cnn_learner(data, models.vgg16_bn, metrics=accuracy)
learn_vgg16.fit(epochs = 10)

In [ ]:
test['vgg16'] = test['files'].apply(lambda x: predict_benign_or_malignant(learn_vgg16, x))
test['vgg16'] = test['vgg16'].astype('str')
test.head()

In [ ]:
accuracy_score(test['target'], test['vgg16'])

In [ ]:
confusion_matrix(test['target'], test['vgg16'])